<a href="https://colab.research.google.com/github/shalvin-shabu/legal-assistant/blob/main/L4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade datasets

In [2]:
#Install and import dependencies
!pip install -q transformers sentencepiece

from transformers import T5Tokenizer, T5ForConditionalGeneration, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import Dataset
import torch

In [3]:
 # Prepare a small toy dataset for legal → simplified text
data = {
    "input_text": [
        "simplify: The lessee shall remit payment within thirty (30) days of receipt.",
        "simplify: The party of the first part agrees to indemnify the party of the second part.",
        "simplify: Notwithstanding the foregoing provisions, either party may terminate the agreement.",
        "simplify: The aforementioned clause shall apply only in cases of gross negligence.",
        "simplify: Hereinafter, the buyer shall be referred to as the purchaser.",
        "simplify: The agreement shall be binding upon and inure to the benefit of the parties hereto.",
        "simplify: The supplier shall not be liable for incidental or consequential damages.",
        "simplify: The contractor must furnish all labor, materials, and equipment necessary for completion.",
        "simplify: The parties agree to resolve disputes through arbitration in lieu of litigation.",
        "simplify: The obligation shall survive the termination of this agreement.",
        "simplify: The tenant shall not assign the lease without prior written consent of the landlord.",
        "simplify: All notices shall be delivered in writing and deemed received upon delivery.",
        "simplify: The borrower shall repay the loan with interest as outlined in the schedule.",
        "simplify: In the event of default, the lender may repossess the collateral.",
        "simplify: This contract supersedes all prior agreements and understandings.",
        "simplify: The license is granted for a non-transferable, non-exclusive use.",
        "simplify: Any provision found unenforceable shall not affect the remaining terms.",
        "simplify: The employee shall maintain confidentiality of proprietary information.",
        "simplify: Each party shall bear its own costs and expenses.",
        "simplify: The terms and conditions are subject to change at the sole discretion of the provider.",
        "simplify: The customer shall be responsible for all applicable taxes.",
        "simplify: This agreement may be executed in counterparts, each of which shall be deemed original.",
        "simplify: The failure to enforce any provision shall not constitute a waiver.",
        "simplify: The parties agree to act in good faith and with fair dealing.",
        "simplify: The user must comply with all local, state, and federal laws.",
        "simplify: This agreement shall be governed by the laws of the State of California.",
        "simplify: Upon breach of any provision herein, the contract may be rendered void.",
        "simplify: The defendant shall appear in court within thirty (30) days of notice.",
        "simplify: The company shall not be held liable for any indirect losses incurred.",
        "simplify: The terms set forth herein shall supersede any prior oral agreements.",
        "simplify: All amendments must be in writing and signed by both parties.",
        "simplify: Failure to deliver within the stipulated timeframe constitutes a breach.",
        "simplify: The court retains jurisdiction over all matters arising hereunder.",
        "simplify: The parties acknowledge that time is of the essence in this agreement.",
        "simplify: The claimant must provide written notice of any claim within 10 days.",
        "simplify: Any waiver of rights must be explicitly stated and documented.",
        "simplify: The agreement shall commence on the date of execution.",
        "simplify: The plaintiff is entitled to recover reasonable attorney’s fees.",
        "simplify: All exhibits attached hereto are incorporated by reference.",
        "simplify: The agent shall act within the scope of the authority granted.",
        "simplify: Each party agrees to hold the other harmless against all claims.",
        "simplify: The policy shall be void if any misrepresentation is discovered.",
        "simplify: The contractor shall obtain all required permits before commencing work.",
        "simplify: All obligations under this agreement shall survive its expiration.",
        "simplify: The buyer has the right to inspect the goods prior to acceptance.",
        "simplify: The employee shall return all company property upon termination.",
        "simplify: No third party shall have rights under this agreement.",
        "simplify: The agreement shall terminate automatically upon completion of services.",
        "simplify: The parties agree that this document reflects their entire agreement.",
        "simplify: No oral modification shall be effective unless confirmed in writing.",
        "simplify: The lessee shall be responsible for all maintenance and repairs.",
        "simplify: The agreement shall remain in effect until terminated by either party.",
        "simplify: The supplier warrants the goods for a period of twelve (12) months.",
        "simplify: Any disputes shall be resolved under the rules of arbitration.",
        "simplify: The buyer shall make full payment upon delivery of the goods.",
        "simplify: This agreement may not be assigned without prior written consent.",
        "simplify: The contractor shall adhere to all applicable building codes.",
        "simplify: The tenant must provide written notice before vacating the premises.",
        "simplify: All intellectual property shall remain the sole property of the creator.",
        "simplify: The insurer shall not be liable for damages caused by natural disasters.",
        "simplify: The obligation to pay taxes rests solely with the buyer.",
        "simplify: The terms herein shall be binding upon all successors and assigns.",
        "simplify: The borrower shall repay the loan according to the payment schedule.",
        "simplify: The contract shall be void if either party engages in fraud.",
        "simplify: The employee shall not disclose confidential information to third parties.",
        "simplify: The parties agree to attempt mediation before litigation.",
        "simplify: No party shall be held liable for indirect or incidental damages.",
        "simplify: The service provider must notify the client of any delays.",
        "simplify: The license is valid for personal use only and may not be shared.",
        "simplify: The policyholder must file a claim within 10 days of the incident.",
        "simplify: All deliveries shall be made during normal business hours.",
        "simplify: The agreement may be amended only by mutual written consent.",
        "simplify: Each party shall bear its own legal costs and expenses.",
        "simplify: The lease shall renew automatically unless terminated in writing.",
        "simplify: The agreement takes effect upon the last signature date.",
        "simplify: The agreement shall be terminated upon mutual consent of the parties.",
        "simplify: The contractor shall complete the work within the stipulated time.",
        "simplify: Any breach of this agreement shall result in immediate termination.",
        "simplify: The employee is entitled to severance pay upon termination.",
        "simplify: All terms and conditions shall be interpreted in accordance with applicable law.",
        "simplify: The parties shall act in good faith in the execution of this agreement.",
        "simplify: The warranty does not cover damage due to misuse.",
        "simplify: The lessee shall obtain prior approval before modifying the premises.",
        "simplify: The user shall comply with all relevant data protection laws.",
        "simplify: The client agrees to the terms outlined in the service level agreement.",
        "simplify: The vendor shall provide support for a period of six (6) months.",
        "simplify: All invoices are due within fifteen (15) days of receipt.",
        "simplify: The agreement constitutes the entire understanding between the parties.",
        "simplify: Either party may request termination with thirty (30) days’ notice.",
        "simplify: The tenant shall not engage in unlawful activity on the premises.",
        "simplify: The buyer shall have the right to return defective goods within 7 days.",
        "simplify: Any delay in performance shall be communicated promptly.",
        "simplify: The landlord may inspect the premises with reasonable notice.",
        "simplify: The policy shall become effective upon acceptance by the insurer.",
        "simplify: No agency, partnership, or joint venture is created by this agreement.",
        "simplify: The payment shall be made in United States Dollars (USD).",
        "simplify: All communications shall be in writing and delivered electronically.",
        "simplify: The license is revocable at the sole discretion of the licensor.",
        "simplify: This clause shall survive the expiration of the agreement.",
        "simplify: The governing law shall be the laws of the Republic of India.",
        "simplify: The supplier shall maintain adequate insurance coverage.",
        "simplify: The parties shall not disclose confidential information.",
        "simplify: The contract shall not be construed to confer rights on third parties.",
        "simplify: The seller represents that all goods are free of liens and encumbrances.",
        "simplify: The lessee agrees to pay a security deposit prior to occupancy.",
        "simplify: The customer shall be responsible for maintaining their login credentials.",
        "simplify: The buyer must inspect all goods upon receipt.",
        "simplify: The data shall be stored in compliance with regulatory standards.",
        "simplify: All documents must be signed and dated by both parties.",
        "simplify: The user agrees to abide by the platform’s community guidelines.",
        "simplify: The contract shall be deemed valid upon digital signature.",
        "simplify: The company reserves the right to update the policy at any time.",
        "simplify: Failure to perform any obligation shall constitute a default.",
        "simplify: The party breaching this agreement shall be liable for damages.",
        "simplify: The court shall have exclusive jurisdiction over any dispute.",
        "simplify: The obligations of the parties are set forth in this agreement.",
        "simplify: Neither party shall be liable for force majeure events.",
        "simplify: All changes to this agreement must be in writing and signed.",
        "simplify: The borrower must not default on any scheduled payments.",
        "simplify: The agreement shall be interpreted without regard to its drafter.",
        "simplify: The client agrees to indemnify the consultant for all losses.",
        "simplify: The policy shall renew automatically unless canceled.",
        "simplify: The parties agree to use electronic signatures for execution.",
        "simplify: The employer shall provide a safe working environment.",
        "simplify: The employee must report any workplace accidents immediately.",
        "simplify: The renter shall not sublease the property without permission.",
        "simplify: The obligation to maintain records lasts for five years.",
        "simplify: The provider may suspend service due to non-payment.",
        "simplify: The agreement becomes null and void upon breach.",
        "simplify: The insured must disclose all material facts.",
        "simplify: The product is covered under warranty for one year.",
        "simplify: The contract binds the parties and their legal heirs.",
        "simplify: The payment plan must be strictly followed.",
        "simplify: The vendor must replace any defective parts.",
        "simplify: The user is responsible for all activity under their account.",
        "simplify: The company may change the terms at any time.",
        "simplify: The tenant is not allowed to keep pets unless stated.",
        "simplify: The agreement covers only the items listed herein.",
        "simplify: The court may grant injunctive relief if necessary.",
        "simplify: The provider must keep all client data confidential.",
        "simplify: The employee must sign a non-compete agreement.",
        "simplify: Any unused leave will not be carried forward.",
        "simplify: Termination must be communicated in writing.",
        "simplify: The organization is exempt from certain taxes.",
        "simplify: The employee is entitled to one paid leave per month.",
        "simplify: All repairs must be completed within 10 business days.",
        "simplify: The service must meet the specifications stated.",
        "simplify: The customer must keep receipts as proof of purchase.",
        "simplify: The parties must resolve conflicts in good faith.",
        "simplify: The license will expire in one year from issue date.",
        "simplify: The document must be notarized to be valid.",
        "simplify: The customer must activate the service within 30 days.",
        "simplify: The agreement is governed by international arbitration rules.",
        "simplify: The company is not responsible for delays caused by third parties.",
        "simplify: The employee must not share internal passwords.",
        "simplify: The organization will report suspicious activity to authorities.",
        "simplify: The service shall begin on the agreed date."


    ],
    "target_text": [
        "The tenant must pay within 30 days.",
        "The first party promises to protect the second party.",
        "Either side can end the agreement even with earlier rules.",
        "This rule applies only if someone was very careless.",
        "From now on, the buyer will be called the purchaser.",
        "This agreement applies to and benefits both parties.",
        "The supplier isn’t responsible for extra or indirect damages.",
        "The builder must provide all tools, workers, and materials.",
        "The sides agree to solve issues using arbitration, not court.",
        "This duty continues even after the agreement ends.",
        "The tenant can’t transfer the lease unless the landlord agrees in writing.",
        "All messages must be written and are received when delivered.",
        "The borrower will repay the loan with interest as scheduled.",
        "If the borrower defaults, the lender can take back the item.",
        "This contract replaces any earlier deals or promises.",
        "The license is for personal use and cannot be shared.",
        "If one rule is invalid, the others still apply.",
        "The employee must keep company information secret.",
        "Each side will pay its own fees.",
        "The provider can change the rules anytime.",
        "The customer must pay any required taxes.",
        "This agreement can be signed in parts, and each counts.",
        "Not using a rule once doesn’t mean it’s gone forever.",
        "Both sides promise to be honest and fair.",
        "The user must follow all laws.",
        "This agreement follows California law.",
        "If any rule is broken, the contract can be canceled.",
        "The accused must come to court within 30 days of getting notice.",
        "The company isn’t responsible for losses it didn’t directly cause.",
        "These terms cancel any earlier spoken agreements.",
        "Changes must be written and signed by both sides.",
        "If delivery is late, it breaks the agreement.",
        "The court can handle any issues from this agreement.",
        "Both sides agree that timing is very important.",
        "The person making a claim must send a letter within 10 days.",
        "Giving up any right must be clearly written down.",
        "The agreement starts when both sides sign it.",
        "The person who sues can get lawyer costs paid back.",
        "Any documents mentioned are part of this agreement.",
        "The agent can only do what they’re allowed to do.",
        "Each side agrees to protect the other from legal problems.",
        "If someone lies, the insurance becomes invalid.",
        "The builder must get all permits before starting.",
        "Duties continue even after the agreement ends.",
        "The buyer can check the goods before accepting them.",
        "The employee must give back company items when they leave.",
        "No outsider has rights from this agreement.",
        "This agreement ends when the work is done.",
        "This document is the full agreement between both sides.",
        "Verbal changes don't count unless written down.",
        "The renter must fix and maintain everything.",
        "This agreement stays valid until one side ends it.",
        "The seller promises the items will work for 12 months.",
        "Arguments will be settled using arbitration.",
        "The buyer must pay in full when they receive the goods.",
        "You can't transfer this agreement without written permission.",
        "The builder must follow building laws and rules.",
        "The renter must write a notice before leaving.",
        "The creator keeps full ownership of their work.",
        "The insurance doesn't cover damage from natural events.",
        "The buyer alone is responsible for paying taxes.",
        "These rules apply to future owners and people involved.",
        "The borrower must follow the loan's payment plan.",
        "If anyone lies, the agreement is canceled.",
        "The worker must keep information private.",
        "Both sides agree to try solving issues before going to court.",
        "No one is responsible for accidental or indirect harm.",
        "The service provider must tell the client about any delays.",
        "This license is just for one person and can't be shared.",
        "The person must report the issue within 10 days.",
        "Deliveries will only happen during normal business hours.",
        "Only both sides together can make changes in writing.",
        "Each side pays for its own legal expenses.",
        "The lease will renew unless one side ends it in writing.",
        "The deal starts on the day both people sign it.",
        "Both sides can cancel this agreement together.",
        "The contractor must finish on time.",
        "If someone breaks the rules, the agreement ends immediately.",
        "The employee will get money when fired.",
        "The rules must follow local laws.",
        "Both sides must act honestly when using this agreement.",
        "Damage from misuse is not covered.",
        "The renter must ask before making changes.",
        "The user must follow data privacy laws.",
        "The client agrees to the service terms.",
        "The seller will help for six months.",
        "Invoices must be paid within 15 days.",
        "This document is the whole deal between both sides.",
        "Either side can end this with 30 days’ notice.",
        "The renter can’t do illegal things there.",
        "The buyer can return faulty items within a week.",
        "Delays must be shared quickly.",
        "The landlord can check the property with notice.",
        "The policy starts when the insurance accepts it.",
        "This agreement does not create a business partnership.",
        "Payment must be in US dollars.",
        "All communication must be sent in writing.",
        "The license can be taken back by the owner anytime.",
        "This rule still applies after the agreement ends.",
        "The laws of India will apply to this deal.",
        "The seller must have insurance.",
        "Neither side will share private info.",
        "This contract gives no rights to others.",
        "The seller promises items are not owed to anyone else.",
        "The renter must pay a deposit before moving in.",
        "The customer must keep their password safe.",
        "The buyer should check items when they arrive.",
        "Data must be stored properly by law.",
        "Both people must sign and date papers.",
        "Users must follow the platform’s rules.",
        "The contract is valid once signed online.",
        "The company can change the rules anytime.",
        "Not doing a task means breaking the contract.",
        "Whoever breaks the deal must pay for the harm.",
        "Only this court will handle any fights over this.",
        "The contract explains each person’s duties.",
        "No one is at fault for things beyond control.",
        "Only written and signed changes are valid.",
        "The borrower must pay on time.",
        "It doesn't matter who wrote the contract.",
        "The client must protect the consultant from losses.",
        "The policy renews unless someone cancels.",
        "They can sign it digitally.",
        "The company must keep workers safe.",
        "The worker must report accidents right away.",
        "You can't rent out the place unless allowed.",
        "You must keep records for five years.",
        "They can stop service if you don’t pay.",
        "If the rules are broken, the deal is off.",
        "The insured must tell all important facts.",
        "The item has a one-year warranty.",
        "The agreement applies to family too.",
        "The payment plan must be followed.",
        "The seller must fix bad parts.",
        "The user is responsible for their account.",
        "The company can change the rules anytime.",
        "No pets are allowed unless written.",
        "The agreement only covers what is listed.",
        "The court can stop someone if needed.",
        "The provider must keep client info secret.",
        "The employee must not work for rivals.",
        "Unused leave doesn’t carry over.",
        "Firing must be in writing.",
        "The group doesn’t have to pay some taxes.",
        "The worker gets one paid leave monthly.",
        "Fixes must be done in 10 business days.",
        "The service must match the promise.",
        "Keep receipts to show you bought something.",
        "They must try to solve problems fairly.",
        "The license ends one year after it’s given.",
        "The paper needs a notary stamp.",
        "You must start using the service in 30 days.",
        "International rules will apply to this agreement.",
        "The company isn't to blame if others cause delays.",
        "The worker can’t share passwords.",
        "The group will report anything suspicious.",
        "The service will start on the agreed day."

  ]
}

In [4]:
dataset = Dataset.from_dict(data)
train_test = dataset.train_test_split(test_size=0.2)
train_dataset = train_test['train']
eval_dataset = train_test['test']



In [5]:
#Tokenizer & model
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained("t5-small", use_fast=True)
model = T5ForConditionalGeneration.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [6]:
#Preprocessing function
def preprocess(batch):
    inputs = tokenizer(batch["input_text"], padding="max_length", truncation=True, max_length=128)
    targets = tokenizer(batch["target_text"], padding="max_length", truncation=True, max_length=128)
    inputs["labels"] = targets["input_ids"]
    return inputs

train_dataset = train_dataset.map(preprocess, batched=True)
eval_dataset = eval_dataset.map(preprocess, batched=True)


# Remove text columns after tokenization
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
eval_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Map:   0%|          | 0/125 [00:00<?, ? examples/s]

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

In [7]:
#Training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./legal_t5_model",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=50,
    logging_steps=10,
    save_steps=50,
    eval_strategy="epoch",
    save_total_limit=1,
    predict_with_generate=True,
    fp16=False,
    report_to="none"
)

In [8]:
# Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer
)


/tmp/ipython-input-8-2052438186.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [9]:
# Train
trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,0.928600,0.436342
2,0.423200,0.328073
3,0.321500,0.301184
4,0.296800,0.263119
5,0.244700,0.215613
6,0.188300,0.171225
7,0.185600,0.159353
8,0.128200,0.155924
9,0.159900,0.153803
10,0.131800,0.153186


KeyboardInterrupt: 

In [ ]:
# Save locally
model.save_pretrained("./legal_t5_model")
tokenizer.save_pretrained("./legal_t5_model")

# Zip it
!zip -r legal_t5_model.zip legal_t5_model

# Download it
from google.colab import files
files.download("legal_t5_model.zip")


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("./legal_t5_model")
model = T5ForConditionalGeneration.from_pretrained("./legal_t5_model")

input_text = "simplify: The supplier warrants the goods for a period of twelve (12) months."
inputs = tokenizer(input_text, return_tensors="pt").to(model.device)

outputs = model.generate(
    **inputs,
    max_new_tokens=64,
    do_sample=True,
    top_k=50,
    temperature=0.9,
    repetition_penalty=1.2,
    early_stopping=True,
    num_return_sequences=1
)

print("Simplified:", tokenizer.decode(outputs[0], skip_special_tokens=True))


# **English to Malayalam model**

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, MarianTokenizer, MarianMTModel

# Load your fine-tuned simplification model
tokenizer = T5Tokenizer.from_pretrained("./legal_t5_model")
model = T5ForConditionalGeneration.from_pretrained("./legal_t5_model")

# Input text for simplification
input_text = "simplify: The supplier warrants the goods for a period of twelve (12) months."
inputs = tokenizer(input_text, return_tensors="pt").to(model.device)

# Generate simplified English output
outputs = model.generate(
    **inputs,
    max_new_tokens=64,
    do_sample=True,
    top_k=50,
    temperature=0.9,
    repetition_penalty=1.2,
    early_stopping=True,
    num_return_sequences=1
)

# Decode and store the simplified output
simplified_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Simplified:", simplified_text)



In [ ]:
# Load the English-to-Malayalam translation model
translator_model_name = "Helsinki-NLP/opus-mt-en-ml"
translator_tokenizer = MarianTokenizer.from_pretrained(translator_model_name)
translator_model = MarianMTModel.from_pretrained(translator_model_name)

# Translate the simplified text to Malayalam
translation_inputs = translator_tokenizer(simplified_text, return_tensors="pt", padding=True, truncation=True)
translated = translator_model.generate(**translation_inputs)
translated_text = translator_tokenizer.decode(translated[0], skip_special_tokens=True)
print("Malayalam Translation:", translated_text)


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, MarianTokenizer, MarianMTModel

# Load your fine-tuned simplification model
tokenizer = T5Tokenizer.from_pretrained("./legal_t5_model")
model = T5ForConditionalGeneration.from_pretrained("./legal_t5_model")

# Input text for simplification
input_text = "simplify:  The service provider must notify the client of any delays."
inputs = tokenizer(input_text, return_tensors="pt").to(model.device)

# Generate simplified English output
outputs = model.generate(
    **inputs,
    max_new_tokens=64,
    do_sample=True,
    top_k=50,
    temperature=0.9,
    repetition_penalty=1.2,
    early_stopping=True,
    num_return_sequences=1
)

# Decode and store the simplified output
simplified_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Simplified:", simplified_text)


In [ ]:
# Load the English-to-Malayalam translation model
translator_model_name = "Helsinki-NLP/opus-mt-en-ml"
translator_tokenizer = MarianTokenizer.from_pretrained(translator_model_name)
translator_model = MarianMTModel.from_pretrained(translator_model_name)

# Translate the simplified text to Malayalam
translation_inputs = translator_tokenizer(simplified_text, return_tensors="pt", padding=True, truncation=True)
translated = translator_model.generate(**translation_inputs)
translated_text = translator_tokenizer.decode(translated[0], skip_special_tokens=True)
print("Malayalam Translation:", translated_text)


# **Interface**

In [ ]:
!pip install streamlit streamlit_jupyter


In [ ]:
# Install required packages
!pip install -q pyngrok transformers

NGROK_AUTH_TOKEN = "2ytowG0mJPc9iMPf1jWZoq2N2rV_3LVb3deVBLxprh9qGrp4c"  #  Replace this line

from pyngrok import ngrok
!ngrok config add-authtoken {NGROK_AUTH_TOKEN}


In [ ]:
%%writefile legal_simplifier_app.py

import streamlit as st
from transformers import T5Tokenizer, T5ForConditionalGeneration, MarianTokenizer, MarianMTModel

# Set Streamlit page config
st.set_page_config(page_title="Legal Text Simplifier", layout="centered")

st.title(" AI Legal Assistant")
st.write("Simplifies legal clauses and translates them to Malayalam.")

# Load models only once
@st.cache_resource
def load_models():
    # Load simplification model
    simplifier_tokenizer = T5Tokenizer.from_pretrained("./legal_t5_model")
    simplifier_model = T5ForConditionalGeneration.from_pretrained("./legal_t5_model")

    # Load English → Malayalam translator
    translator_model_name = "Helsinki-NLP/opus-mt-en-ml"
    translator_tokenizer = MarianTokenizer.from_pretrained(translator_model_name)
    translator_model = MarianMTModel.from_pretrained(translator_model_name)

    return simplifier_tokenizer, simplifier_model, translator_tokenizer, translator_model

# Load all models
simplifier_tokenizer, simplifier_model, translator_tokenizer, translator_model = load_models()

# User input
user_input = st.text_area(" Enter legal text to simplify:", height=150)

if st.button("Simplify and Translate"):
    if user_input.strip() == "":
        st.warning(" Please enter some legal text.")
    else:
        with st.spinner("Simplifying..."):
            # Prepare input for T5
            input_text = "simplify: " + user_input
            inputs = simplifier_tokenizer(input_text, return_tensors="pt").to(simplifier_model.device)

            # Generate simplified English text
            outputs = simplifier_model.generate(
                **inputs,
                max_new_tokens=64,
                do_sample=True,
                top_k=50,
                temperature=0.9,
                repetition_penalty=1.2,
                early_stopping=True,
                num_return_sequences=1
            )
            simplified_text = simplifier_tokenizer.decode(outputs[0], skip_special_tokens=True)

        st.success("Simplified Text (in English):")
        st.markdown(f"Assistant: {simplified_text}")

        with st.spinner("Translating to Malayalam..."):
            # Translate simplified text to Malayalam
            translation_inputs = translator_tokenizer(simplified_text, return_tensors="pt", padding=True, truncation=True)
            translated = translator_model.generate(**translation_inputs)
            translated_text = translator_tokenizer.decode(translated[0], skip_special_tokens=True)

        st.success("Translation (in Malayalam):")
        st.markdown(f"Assistant: {translated_text}")



In [ ]:
from pyngrok import ngrok
import time
import os

# Kill old tunnels
ngrok.kill()

# Start the streamlit app
os.system("streamlit run legal_simplifier_app.py &")
time.sleep(5)  # Wait for app to boot

# Tunnel the port
public_url = ngrok.connect(addr=8501)
print(f"🌐 Your app is live at: {public_url}")


In [ ]:
ngrok.kill()

input for Testing

"simplify: The lessee shall be responsible for all maintenance and repairs."

"simplify: The agreement shall remain in effect until terminated by either party."

"simplify: The buyer shall make full payment upon delivery of the goods."

"simplify: All amendments must be in writing and signed by both parties."

"simplify: The employee shall not disclose confidential information to third parties."

"simplify: The contract shall be void if either party engages in fraud."

"simplify: The insurer shall not be liable for damages caused by natural disasters."

"simplify: The borrower shall repay the loan according to the payment schedule."

"simplify: Any disputes shall be resolved under the rules of arbitration."

"simplify: The supplier warrants the goods for a period of twelve (12) months."

"simplify: The tenant must provide written notice before vacating the premises."

"simplify: The policyholder must file a claim within 10 days of the incident."

"simplify: The company reserves the right to update the policy at any time."

"simplify: The service provider must notify the client of any delays."

"simplify: The agreement takes effect upon the last signature date."

"simplify: The contractor shall adhere to all applicable building codes."

"simplify: Each party agrees to hold the other harmless against all claims."

"simplify: The user shall comply with all relevant data protection laws."

"simplify: No party shall be held liable for indirect or incidental damages."

"simplify: The parties agree to attempt mediation before litigation."

